In [ ]:
import json
import os
import string
import pandas as pd
import plotly.graph_objects as go
import datautils

# pd.set_option('max_columns', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = None
# pd.set_option('max_row', None)

In [ ]:
lstbg, lstfg = datautils.loadAllFiles()

In [ ]:
dfreels = datautils.genDataframeModeReels(lstbg, lstfg)

dfreels

In [ ]:
wins = datautils.genDataframeWinnings(lstbg, lstfg)

wins.sort_values(by='winnings')

In [ ]:
wins2 = datautils.genDataframeWinningsEx(lstbg, lstfg, [0, 1, 2, 3, 5, 10, 20, 30, 50, 100, 200, 300, 500, 1000])

datautils.showWinningsPie(wins2)

In [ ]:
dfsymbols = datautils.genDataframeSymbols(lstbg, lstfg)

dfsymbols.sort_values(by='Symbol')

In [ ]:
dfsymbolwin = datautils.genSymbolWins(lstbg, lstfg)

lst = list(dfsymbolwin.columns)
lst.sort()
dfsymbolwin[lst].sort_values(by=' Symbol')

In [ ]:
dfmystery = datautils.genMysterySymbolReveal(lstbg, lstfg)

dfmystery.sort_values(by='MysterySymbolReveal')

In [ ]:
dfMysterySymbolSize = datautils.genMysterySymbolSize(lstbg, lstfg)

dfMysterySymbolSize

In [ ]:
# lstm = datautils.findMystery(lstbg)

# lstbg[1]